In [ ]:
import sys

# Load diabetes dataset from sklearn
from sklearn.datasets import load_iris

import dc_check.logger as log
log.add(sink=sys.stderr, level="DEBUG")


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = "mnist"
model_name = "LeNet"

if dataset == "mnist":
    rule_matrix = {
                1: [7],
                2: [7],
                3: [8],
                4: [4],
                5: [6],
                6: [5],
                7: [1, 2],
                8: [3],
                9: [7],
                0: [0]
            }
    
    # Define transforms for the dataset
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    # Load the MNIST dataset
    train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)


    X_train, y_train = train_dataset.data.type(torch.FloatTensor), train_dataset.targets.type(torch.LongTensor)
    X_test, y_test = test_dataset.data.type(torch.FloatTensor), test_dataset.targets.type(torch.LongTensor)

elif dataset == "cifar":
    rule_matrix = {
                0: [2],   # airplane (unchanged)
                1: [9],   # automobile -> truck
                2: [9],   # bird (unchanged)
                3: [5],   # cat -> automobile
                4: [5,7],   # deer (unchanged)
                5: [3, 4],   # dog -> cat
                6: [6],   # frog (unchanged)
                7: [5],   # horse -> dog
                8: [7],   # ship (unchanged)
                9: [9],   # truck -> horse
            }
    
    # Define transforms for the dataset
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # Load the CIFAR-10 dataset
    train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
    test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
    X_train, y_train = train_dataset.data, train_dataset.targets
    X_test, y_test = test_dataset.data, test_dataset.targets

else:
    raise ValueError("Invalid dataset!")

print(y_train.type())

In [ ]:

import dc_check.plugins.core.models.image_nets as im_nets
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# Instantiate the neural network 
if dataset == 'cifar':
    if model_name == 'LeNet':
        model = im_nets.LeNet(num_classes=10).to(DEVICE)
    if model_name == 'ResNet':
        model = im_nets.ResNet18().to(DEVICE)
elif dataset == 'mnist':
    if model_name == 'LeNet':
        model = im_nets.LeNetMNIST(num_classes=10).to(DEVICE)
    if model_name == 'ResNet':
        model = im_nets.ResNet18MNIST().to(DEVICE)


# creating our optimizer and loss function object
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)


X_train = torch.Tensor(X_train).unsqueeze(1)
X_test = torch.Tensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)


# Passing to DataLoader
class IndexedDataset(TensorDataset):

    def __getitem__(self, index):
        data, target = TensorDataset.__getitem__(self, index)
        return data, target, index
    
    def __len__(self):
        return self.tensors[0].shape[0]

    
dataset_1 = IndexedDataset(X_train, y_train)

dataloader = DataLoader(
    dataset_1,
    batch_size=64,
    shuffle=True,
    num_workers=0,
)

dataloader_unshuffled = DataLoader(
    dataset_1,
    batch_size=64,
    shuffle=False,
    num_workers=0,
)


In [ ]:
# dc_check absolute
from dc_check.plugins import Plugins

Plugins().list()


In [ ]:

hcm = Plugins().get(
    "data_iq",
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    lr=learning_rate,
    epochs=10,
    total_samples=len(X_train),
    num_classes=10,
    logging_interval=1,
)
hcm.fit(
    dataloader=dataloader,
    dataloader_unshuffled=dataloader_unshuffled,
)

In [ ]:
hcm_scores = hcm.compute_scores()
print(hcm_scores)
hcm.plot_scores()
hcm.plot_scores(axis=0)
hcm.plot_scores(axis=1)